In [1]:
# imports needed for this notebook: Run this cell
from collections import deque
from queue import PriorityQueue

# 15-Puzzle

In dieser Aufgabe möchten wir das 15-Puzzle (https://de.wikipedia.org/wiki/15-Puzzle) implementieren und verschiedene Suchalgorithmen darauf anwenden. 

Das Spiel besteht aus 15 Kacheln, von 1 bis 15 durchnummeriert, die auf den 16 Feldern eines Vier-mal-vier-Quadrats angebracht sind. Ein Feld (das „Loch“) bleibt also frei. Eine (vertikal oder horizontal) benachbarte Kachel kann jeweils in das freie Feld hineingeschoben werden. Die Aufgabe besteht nun darin, durch Verschieben der Kacheln die Zahlen von 1 bis 15 aufsteigend anzuordnen.

Wir stellen eine Position als Tupel dar, indem wir alle 4 Reihen nacheinander aufschreiben. Das Loch kennzeichnen wir mit 0. Ein Beispiel für eine Position wäre folgende:



In [2]:
start_pos = (5, 1, 3, 4, 6, 9, 7, 8, 2, 10, 0, 11, 13, 14, 15, 12)

Für eine bessere Lesbarkeit verwenden wir eine ASCII-Darstellung des Zustands:

In [3]:
def print_position(position):
    """
    Print the ascii representation in human-readable form
    Arguments: 
    position: 16-tuple containing numbers from 0 to 15
    return: None
    """
    size = 4
    two_d_pos = [list(position[i:i + size]) for i in range(0, len(position), size)]
    print("+" + "-" * size * 3 + "+")
    for row in two_d_pos:
        row_str = "|"
        for elem in row:
            if elem == 0:
                row_str += "   "
            else:
                row_str += str(elem) + " " * (3 - len(str(elem)))
        row_str += "|"
        print(row_str)
    print("+" + "-" * size * 3 + "+")

Unsere oben definierte Startposition sieht damit folgendermaßen aus:

In [4]:
print_position(start_pos)

+------------+
|5  1  3  4  |
|6  9  7  8  |
|2  10    11 |
|13 14 15 12 |
+------------+


Unsere Zielposition ist folgendermaßen codiert:

In [5]:
goal_pos = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
print_position(goal_pos)

+------------+
|1  2  3  4  |
|5  6  7  8  |
|9  10 11 12 |
|13 14 15    |
+------------+


## a) Implementierung von get_moves()
Um die Suchalgorithmen aus der Vorlesung anzuwenden, müssen wir zunächst in
der Lage sein, einen Knoten zu expandieren. Genauer gesagt müssen wir in
einer gegebenen Position alle Positionen berechnen, die man in einem Zug
erreichen kann. Ist das Loch in einen der vier mittleren Positionen, gibt es 4
mögliche Züge. Falls das Loch am Rand oder in der Ecke ist, gibt es jeweils 3
oder nur 2 mögliche Züge. Implementiere die Berechnung der neuen Positionen in
der Funktion `get_moves(position)`. Beachte dabei die Docstring-Kommentare,
die die Ein- und Ausgabe spezifizieren.

**Hinweise:**
- Um den Index des Lochs zu erhalten, verwende `pos.index(0)`
- Du musst 4 Fallunterscheidungen betrachten, jeweils ob das Loch am linken,
rechten, oberen oder unteren Rand sich befindet.
- Benutze den Modulo Operator  `%`  und `>=` bzw. `<` um zu testen, wo sich
die Position des Lochs befindet. Damit kannst du testen, ob ein Zug hier
möglich ist.
- Verwende `new_move = list(pos)`, um eine veränderbare Kopie des aktuellen
Zustands zu erhalten. In diese kannst du dann einen neuen Zug schreiben.
- Durch `moves.append(tuple(new_move))` kannst du den neu erstellten Zug
deiner Liste als Tupel hinzufügen.

In [11]:
BOARD_WIDTH = 4
BOARD_HEIGTH = 4

# 0   1   2   3
# 4   5   6   7
# 8   9  10   11
# 12 13  14   15
# 9 / 4 = 2
# 9 % 4 = 1
# 6 => 2, 5, 7, 10
#
#

def checkIsNeighbour(index):
    return (0 <= index) and (index < BOARD_WIDTH * BOARD_HEIGTH)
    
def getNeighbours(index):
    up = index - BOARD_WIDTH
    left = index - 1 if (index % BOARD_WIDTH != 0) else -1
    right = index + 1 if (index % BOARD_WIDTH != (BOARD_WIDTH - 1)) else -1
    down = index + BOARD_WIDTH
    return list(filter(checkIsNeighbour,  [up, left, right, down]))

def get_moves(pos):
    """
    Compute the reachable positions given the current position `pos` by moving
     one adjacent number into the hole.
    Parameters:
        `pos`: 16-tuple of the current position
    Returns:
        List of 16-tuples containing the reachable positions
    
    """
    moves = []

    index_of_hole = pos.index(0)
    # for element in pos:
    #     if element == 0:
    #         break
    #     index_of_hole = index_of_hole + 1

    for index_of_neigbour in getNeighbours(index_of_hole):
        new_pos = list(pos)

        
        new_pos[index_of_hole] = pos[index_of_neigbour]
        new_pos[index_of_neigbour] = 0
        moves.append(tuple(new_pos))

    return moves

Du kannst hier die Korrektheit an 2 Bespielen überprüfen. Wenn du möchtest,
kannst du noch weitere Beispiele erstellen und den Output von `get_moves` mit
`print_position(pos)` per Hand auf Korrektheit überprüfen. Nur mit einer
richtigen Implementierung von `get_moves` ist es möglich, richtige Ergebnisse
bei den Suchalgorithmen zu erhalten. Teste deshalb ausführlich, ob deine
Implementierung korrekt ist.

In [12]:
# first test position
pos_1 = start_pos
moves_pos_1 = get_moves(pos_1)

correct_moves_pos_1 = [
    (5, 1, 3, 4, 6, 9, 7, 8, 2, 0, 10, 11, 13, 14, 15, 12),
    (5, 1, 3, 4, 6, 9, 7, 8, 2, 10, 11, 0, 13, 14, 15, 12),
    (5, 1, 3, 4, 6, 9, 0, 8, 2, 10, 7, 11, 13, 14, 15, 12),
    (5, 1, 3, 4, 6, 9, 7, 8, 2, 10, 15, 11, 13, 14, 0, 12)
]

print("Testing Position:")
print_position(pos_1)
print("Calculated moves:")

for move in moves_pos_1:
    print_position(move)

assert len(moves_pos_1) == len(correct_moves_pos_1)

for move in moves_pos_1:
    assert move in correct_moves_pos_1

print("All tests passed!\n\n")

# second test position
pos_2 = (5, 1, 3, 4, 6, 9, 7, 8, 2, 10, 11, 0, 13, 14, 15, 12)
moves_pos_2 = get_moves(pos_2)
correct_moves_pos_2 = [
    (5, 1, 3, 4, 6, 9, 7, 8, 2, 10, 0, 11, 13, 14, 15, 12),
    (5, 1, 3, 4, 6, 9, 7, 8, 2, 10, 11, 12, 13, 14, 15, 0),
    (5, 1, 3, 4, 6, 9, 7, 0, 2, 10, 11, 8, 13, 14, 15, 12)
]

print("Testing Position:")
print_position(pos_2)
print("Calculated moves:")

for move in moves_pos_2:
    print_position(move)

assert len(moves_pos_2) == len(correct_moves_pos_2)

for move in moves_pos_2:
    assert move in correct_moves_pos_2

print("All tests passed!")


Testing Position:
+------------+
|5  1  3  4  |
|6  9  7  8  |
|2  10    11 |
|13 14 15 12 |
+------------+
Calculated moves:
+------------+
|5  1  3  4  |
|6  9     8  |
|2  10 7  11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|6  9  7  8  |
|2     10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|6  9  7  8  |
|2  10 11    |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|6  9  7  8  |
|2  10 15 11 |
|13 14    12 |
+------------+
All tests passed!


Testing Position:
+------------+
|5  1  3  4  |
|6  9  7  8  |
|2  10 11    |
|13 14 15 12 |
+------------+
Calculated moves:
+------------+
|5  1  3  4  |
|6  9  7     |
|2  10 11 8  |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|6  9  7  8  |
|2  10    11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|6  9  7  8  |
|2  10 11 12 |
|13 14 15    |
+------------+
All tests passed!


## b) Test auf Goal Position
Als Nächstes müssen wir herausfinden, ob eine Position dem Zielzustand
entspricht. Implementiere dazu die Funktion `is_goal(pos)`.

In [15]:
def is_goal(pos):
    """
    Test if the current position `pos` is the desired goal position
    Parameters:
        `pos`: 16-tuple of the current position
    Returns:
        Boolean: True if `pos`is goal position, False otherwise
    
    """
    return pos == goal_pos

In [103]:
pos = (15, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0)
assert type(is_goal(pos)) == bool

assert is_goal((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)) == True

## c) Implementierung der Breitensuche

Nun implementieren wir den ersten Suchalgorithmus, die Breitensuche (BFS).
In unserem Fall entspricht dies der Uniform-Cost-Search, da wir bei jedem
Zustandsübergang Kosten von 1 haben (wir sind nur interessiert an der Anzahl
der Züge der Lösung). Der allgemeine Algorithmus aus der Vorlesung für
Tree-Search finden wir auf Folie 41. Wir geben ihn hier der Vollständigkeit
halber noch einmal an:

```
function TREE-SEARCH(problem, strategy) returns a solution, or failure
    initialize fringe using the initial state of the problem
    loop do
        if there are no candidates in fringe then return failure
        choose a fringe node for expansion according to strategy (remove node from fringe list)
        if the node is a goal state then return corresponding solution
        else expand the node: add all possible successor nodes to fringe list
    end
```

Für die Breitensuche verwenden wir eine Queue als Datenstruktur.
Eine Implementierung in Python ist `deque()` im Modul `collections`
(https://docs.python.org/3/library/collections.html#collections.deque).
Wir wählen den zuerst hinzugefügten Knoten in der Fringe als Knoten aus,
der expandiert wird.

Dieser Algorithmus ist allerdings nicht performant, da wir eine Baumsuche
durchführen. Hier kann der gleiche Zustand öfters in die Fringe hinzugefügt
und mehrmals expandiert werden. Dies ist extrem ineffizient, da wir diese
Arbeit nur einmal pro Knoten durchführen müssen. Eine Lösung ist die
Graphsuche: Wir merken uns in einer Datenstruktur, welche Knoten wir schon
expandiert haben. Eine Möglichkeit wäre, eine Liste `expanded_nodes = []`
hierfür zu verwenden. Jedoch ist dies ineffizient, da wir öfters testen müssen,
ob ein Element in dieser Liste ist, was jeweils eine lineare Komplexität hat.
Besser ist es, ein dictionary `expanded_nodes = {}` zu nutzen und expandierte
Knoten mit einem `expanded_nodes[pos] = True` zu markieren. Mittels
`pos in expanded_nodes` kann in konstanter Zeit überprüft werden, ob ein
Knoten schon expandiert wurde.

Schließlich möchten wir auch noch unseren Pfad zum Zielzustand ausgeben.
Dazu speichern wir für jeden Zustand seinen Vorgänger, also der Knoten,
auf den `get_moves()` aufgerufen wurde. Dies können wir ebenfalls in einem
dictionary tun.

Der Pseudocode für diesen abgewandelten Algorithmus sieht folgendermaßen aus:

```
function GRAPH-SEARCH(problem, strategy) returns a solution, or failure
    initialize fringe using the initial state of the problem
    initialize predecessors as an empty dictionary
    initialize expanded_nodes as an empty dictionary
    loop do
        if there are no candidates in fringe then return failure
        choose a fringe node for expansion according to strategy (remove node from fringe list)
        if the node is a goal state then return corresponding solution
        else: 
            expand the node
            set expanded_nodes of this node to True
            for each possible successor:
                if the successor is not in expanded_nodes:
                    add it to the fringe
                    select the currently expanded node of this successor as the predecessor
            
    end
    

```

In [104]:
def bfs(start_pos):
    """
    Execute the BFS with the initial position `start_pos`.
    It raises an Assertion Error when the solution cannot be found.
    Parameters:
        `start_pos`: 16-tuple of the current position
    Returns: 
        The tuple (predecessors, num_steps) when the goal position is found.
    """
    expanded_nodes = {}
    predecessors = {}
    queue = deque()
    queue.append(start_pos)
    num_steps = 0
    while len(queue) > 0:
        num_steps += 1
        curr_pos = queue.popleft()
        
        if is_goal(curr_pos):
            return predecessors, num_steps
    
        if expanded_nodes.get(curr_pos):
            continue
        
        expanded_nodes[curr_pos] = True
        
        for next_move in get_moves(curr_pos):
            if not expanded_nodes.get(next_move):
                queue.append(next_move)
                predecessors[next_move] = curr_pos

                    
    raise ValueError("Fringe is empty!")

Die folgende Funktion gibt den Pfad der Lösung aus:

In [105]:
def show_solution(start_pos, final_pos, predecessors):
    path = [final_pos]
    while final_pos != start_pos:
        final_pos = predecessors[final_pos]
        path.append(final_pos)
    for pos in reversed(path):
        print_position(pos)
    print(f"Länge der Lösung: {len(path)}")
    return len(path)

Teste deine Implementierung an folgendem Beispiel:

In [106]:
start_pos = (5, 1, 3, 4, 6, 9, 7, 8, 2, 10, 0, 11, 13, 14, 15, 12)
predecessors, num_steps = bfs(start_pos)
final_pos = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
len_sol = show_solution(start_pos, final_pos, predecessors)
print(f"Anzahl an Steps: {num_steps}")
assert len_sol == 15


+------------+
|5  1  3  4  |
|6  9  7  8  |
|2  10    11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|6  9  7  8  |
|2     10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|6  9  7  8  |
|   2  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|   9  7  8  |
|6  2  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|9     7  8  |
|6  2  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|9  2  7  8  |
|6     10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|9  2  7  8  |
|   6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|   2  7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|   1  3  4  |
|5  2  7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|1     3  4  |
|5  2  7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|1  2  3  4  |
|5     7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+---------

## d) Erste Heuristik für A*
Abhängig von der Reihenfolge von der Liste in `get_moves()` benötigen wir
40000 bis über 100000 Schritte bis der Algorithmus terminiert. Das wollen wir
mit dem A* Algorithmus beschleunigen. Hierfür benötigen wir zunächst eine
Heuristik. Unsere erste Heuristik ist die Anzahl an bisher noch falsch
platzierten Zahlen, wie in der Vorlesung beschrieben. Das Loch zählen wir hier
nicht als Zahl: die korrekte Lösung hat somit 0 falsch platzierte Zahlen,
eine komplett durchmischte Position enthält 15 falsch platzierte Zahlen.
Vervollständige die Funktion `num_wrong_heuristic()`.

In [114]:
def num_wrong_heuristic(pos):
    """
    Computes the number of wrongly placed tiles.
    Parameters:
        `pos`: 16-tuple of the current position
    Returns: 
        Number of wronlgy placed tiles.
    """

    num_wrong = 0

    pos_list = list(pos)
    goal_pos_list = list(goal_pos) 

    
    for i in range(0, len(pos_list)):
        if pos_list[i] == 0:
            continue

        if pos_list[i] != goal_pos_list[i]:
            num_wrong = num_wrong + 1

    return num_wrong

In [115]:
pos = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
assert num_wrong_heuristic(pos) == 0
pos2 = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14, 0)
assert num_wrong_heuristic(pos2) == 2
pos3 = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
assert num_wrong_heuristic(pos3) == 15

## e) A* Implementierung
Nun wollen wir die eben definierte Heuristik nutzen, um den A*-Algorithmus zu
implementieren. Hierbei ist die Fringe eine Priority-Queue, welche in Python
im Modul `queue` als `PriorityQueue` definiert ist (https://docs.python.org/3/library/queue.html).
Um eine neue Position mit einer Priorität in die Priority-Queue hinzuzufügen,
verwenden wir die Methode `queue.put()`. Wir übergeben ein Tupel der Form
```
(Heuristik von pos + Backward Kosten von pos, Backward Kosten von pos, pos).
```
Wir müssen die Backward Kosten als zweiten Eintrag nochmals separat übergeben,
da wir sie für die nächsten Backward Kosten des Nachfolgers benötigen.
Um das Element mit den niedrigsten Kosten aus der Queue zu erhalten,
rufen wir die Methode
```
cost, bw_cost, pos = queue.get()
```
auf. 

Um die Implementierung modular zu halten, übergeben wir die heuristic Funktion
als Parameter. Implementiere die Funktion `a_star()`.

**Hinweis**:
Denk beim Expandieren daran, dass sich die Backward Kosten für die Nachfolger
um 1 erhöht.

In [116]:
def a_star(start_pos, heuristic):
    """
    Execute the A* algorithm with the initial position `start_pos`.
    Uses the heuristic function `heuristic`.
    It raises an Assertion Error when the solution cannot be found.
    Parameters:
        `start_pos`: 16-tuple of the current position
        `heuristic`: Callable, which takes a position as a parameter and
        computes a heuristic value of that.
    Returns: 
        The tuple (predecessors, num_steps) when the goal position is found.
    """
    expanded_nodes = {}
    predecessors = {}
    queue = PriorityQueue()
    queue.put((heuristic(start_pos), 0, start_pos))
    num_steps = 0
    while queue.qsize() > 0:
        num_steps += 1
        cost, bw_cost, curr_pos = queue.get()
        
        if is_goal(curr_pos):
            return predecessors, num_steps
    
        if expanded_nodes.get(curr_pos):
            continue
        
        expanded_nodes[curr_pos] = True
        
        for next_move in get_moves(curr_pos):
            if not expanded_nodes.get(next_move):
                queue.put((heuristic(next_move) + bw_cost, bw_cost + 1, next_move))
                predecessors[next_move] = curr_pos
        
        
    raise ValueError("Fringe is empty!")

In [117]:
start_pos = (5, 1, 3, 4, 6, 9, 7, 8, 2, 10, 0, 11, 13, 14, 15, 12)
predecessors, num_steps = a_star(start_pos, heuristic=num_wrong_heuristic)
final_pos = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
len_sol = show_solution(start_pos, final_pos, predecessors)
print(f"Anzahl an Steps: {num_steps}")
assert len_sol == 15


+------------+
|5  1  3  4  |
|6  9  7  8  |
|2  10    11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|6  9  7  8  |
|2     10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|6  9  7  8  |
|   2  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|   9  7  8  |
|6  2  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|9     7  8  |
|6  2  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|9  2  7  8  |
|6     10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|9  2  7  8  |
|   6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|   2  7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|   1  3  4  |
|5  2  7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|1     3  4  |
|5  2  7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|1  2  3  4  |
|5     7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+---------

## f) Manhattan Heuristic
A* benötigt für das gleiche Suchproblem weniger als 300 Schritte (im Vergleich
zu den über 40000 von BFS). Das ist schon eine deutliche Verbesserung.
Mit einer besseren Heuristik lässt sich das allerdings noch verringern. Die
Kosten werden von der `num_wrong_heuristic` stark nach unten abgeschätzt, da
es im Normalfall deutlich mehr als einen Zug pro falsche Zahl braucht. Eine
bessere Heuristik ist die Manhattan Heuristic: Hier wird für jede Zahl die
Manhattan Distanz (https://de.wikipedia.org/wiki/Manhattan-Metrik) zum
richtigen Ort ermittelt. Implementiere die Funktion `manhattan_heuristic`,
die die Summe der Manhattan Distanzen aller Zahlen zurückgibt (wieder
ignorieren wir das Loch für diese Berechnung).

**Hinweis**:
Es ist sinnvoll, sich eine 2D-Darstellung `goal_2d` der Zielposition zu
speichern. Anschließend kann man über die Elemente der aktuellen Position
iterieren, die Zielkoordinaten der aktuellen Zahl in `goal_2d` nachschauen
und die Manhattan Distanz mithilfe der Funktion `abs` bestimmen. Eine
Skizze mit Stift und Papier ist hier hilfreich.

In [157]:
list_of_goal_coor = [(3,3)]
[list_of_goal_coor.append((i, j)) for i in range(0, BOARD_HEIGTH) for j in range(0, BOARD_WIDTH)]
list_of_goal_coor = list_of_goal_coor[0:(BOARD_HEIGTH * BOARD_WIDTH)] 

print(list_of_goal_coor)


def getDistance(p1, p2):
    """
    Computes the Manhattan distances of two points with same dimension.
    Parameters:
        `p1`: firts point: tuple of coordinates.
        `p2`: second point: tuple of coordinates.
    Returns: 
        the Manhattan distances.
    """
    p1_as_list = list(p1)
    p2_as_list = list(p2)

    dim = len(p1_as_list)
    if not dim == len(p2_as_list):
        raise IndexError("Coordinates have diffrent dimensions.")

    dist = 0
    for i in range(0, dim):
        dist += abs(p1_as_list[i] - p2_as_list[i])
    
    return dist

def getCoordinates(index):
    """
    Computes the the board coordinates of a given board element.
    Parameters:
        `index`: List index of the board element.
    Returns: 
        the board coordinates. as a tupel of (row, column).
    """

    row = index // BOARD_WIDTH
    if row >= BOARD_HEIGTH:
        raise IndexError("Index out of bounds.")
    column = index % BOARD_WIDTH

    return (row, column)


def manhattan_heuristic(pos):
    """
    Computes the sum of the Manhattan distances of every tile
    to its desired position.
    Parameters:
        `pos`: 16-tuple of the current position
    Returns: 
        Sum of the Manhattan distances.
    """
    dist = 0
    
    pos_list = list(pos)
    goal_pos_list = list(goal_pos) 

    
    for i in range(0, len(pos_list)):
        curr_index = pos_list[i]
        if curr_index == 0:
            continue
        goal_coor = list_of_goal_coor[curr_index]
        curr_coor = getCoordinates(i)
        dist += getDistance(goal_coor, curr_coor)

    return dist



[(3, 3), (0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2)]


In [160]:
pos = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
assert manhattan_heuristic(pos) == 0
pos2 = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14, 0)
assert manhattan_heuristic(pos2) == 2
pos3 = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15)
assert manhattan_heuristic(pos3) == 1
pos4 = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
assert manhattan_heuristic(pos4) == 24

Lasse zum Abschluss A* mit der Manhattan Heuristik laufen. Die Anzahl an
steps sollte nun bei ca. 70 liegen. Wir sehen also, dass A* mit einer guten
Heuristik bis zu 1000 Mal schneller laufen kann im Vergleich zur BFS. Je
komplizierter das Suchproblem wird, desto mehr wird eine informierte Suche
und eine gute Heuristik benötigt.

In [161]:
start_pos = (5, 1, 3, 4, 6, 9, 7, 8, 2, 10, 0, 11, 13, 14, 15, 12)
predecessors, num_steps = a_star(start_pos, heuristic=manhattan_heuristic)
final_pos = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
len_sol = show_solution(start_pos, final_pos, predecessors)
print(f"Anzahl an Steps: {num_steps}")

+------------+
|5  1  3  4  |
|6  9  7  8  |
|2  10    11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|6  9  7  8  |
|2     10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|6  9  7  8  |
|   2  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|   9  7  8  |
|6  2  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|9     7  8  |
|6  2  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|9  2  7  8  |
|6     10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|9  2  7  8  |
|   6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|5  1  3  4  |
|   2  7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|   1  3  4  |
|5  2  7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|1     3  4  |
|5  2  7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+------------+
|1  2  3  4  |
|5     7  8  |
|9  6  10 11 |
|13 14 15 12 |
+------------+
+---------